In [1]:
!pip install python-dotenv
!pip install -U torch transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python312\\Scripts\\dotenv.exe' -> 'C:\\Python312\\Scripts\\dotenv.exe.deleteme'



Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 0.0/134.8 kB ? eta -:--:--
     -------------------------------------- 134.8/134.8 kB 7.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 61.0/61.0 kB ? eta 0:00:00
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB ? eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB ? eta 0:00:00
   ---------------------------------------- 0.0/198.5 MB ? eta -:--:--
   ---------------------------------------- 2.2/198.5 MB 45.5 MB/s eta 0:00:05
    --------------------------------------- 3.9/198.5 MB 41.7 MB/s eta 0:00:05
   - -------------------------------------- 5.5/198.5 MB 38.9 MB/s eta 0:00:05
   - --------------------------

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Python312\\share'
Consider using the `--user` option or check the permissions.



# Gemma Model

In [5]:
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from dotenv import load_dotenv
load_dotenv()

ACCESS_TOKEN = os.getenv("ACCESS_TOKEN") # reads .env file with ACCESS_TOKEN=<your hugging face access token>

model_id = "google/gemma-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id, token=ACCESS_TOKEN)
quantization_config = BitsAndBytesConfig(load_in_4bit=True, 
                                         bnb_4bit_use_double_quant=True,
                                         bnb_4bit_compute_dtype=torch.bfloat16)

model = AutoModelForCausalLM.from_pretrained(model_id, 
                                             device_map="auto", 
                                             quantization_config=quantization_config,
                                             token=ACCESS_TOKEN)
model.eval()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"using {device}")




OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-2b-it.
401 Client Error. (Request ID: Root=1-6616b670-653190c74f8cd034658a0be9;f89046aa-4c34-4bdc-b425-4a59e9daf1fb)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b-it/resolve/main/config.json.
Repo model google/gemma-2b-it is gated. You must be authenticated to access it.

In [ ]:
def inference(question: str, context: str):

    if context == None or context == "":
        prompt = f"""Give a detailed answer to the following question. Question: {question}"""
    else:
        prompt = f"""Using the information contained in the context, give a detailed answer to the question.
            Context: {context}.
            Question: {question}"""
    chat = [
        {"role": "user", "content": prompt},
        # { "role": "model", "content": "Recurrent Attention (RAG)** is a novel neural network architecture specifically designed" }
    ]
    formatted_prompt = tokenizer.apply_chat_template(
        chat,
        tokenize=False,
        add_generation_prompt=True,
    )
    inputs = tokenizer.encode(
        formatted_prompt, add_special_tokens=False, return_tensors="pt"
    ).to(device)
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs,
            max_new_tokens=250,
            do_sample=False,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    response = response[len(formatted_prompt) :]  # remove input prompt from reponse
    response = response.replace("<eos>", "")  # remove eos token
    return response


question = "What is a transformer?"
print(inference(question=question, context=""))

# Document Loading and Splitting

In [ ]:
!pip install pypdf
!pip install tiktoken

In [8]:
from langchain.document_loaders import PyPDFLoader

loaders = [
    PyPDFLoader("/home/eversberg/Downloads/1706.03762.pdf"),
    PyPDFLoader("/home/eversberg/Downloads/2005.11401.pdf"),
]
pages = []
for loader in loaders:
    pages.extend(loader.load())

In [9]:
from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=128, chunk_overlap=12)
docs = text_splitter.split_documents(pages)

In [ ]:
print(docs[0].page_content)

# Embeddings and Vector Store

In [ ]:
!pip install -U sentence-transformers

In [ ]:

import numpy as np
from langchain_community.embeddings import (
    HuggingFaceEmbeddings
)
encoder = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L12-v2', model_kwargs = {'device': "cpu"})


In [ ]:
embeddings1 = encoder.embed_query("RAG")
embeddings2 = encoder.embed_query(docs[0].page_content)
print(np.dot(embeddings1, embeddings2))

In [ ]:
!pip install faiss-cpu

In [13]:
from langchain.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
faiss_db = FAISS.from_documents(docs, encoder, distance_strategy=DistanceStrategy.DOT_PRODUCT)

In [ ]:
question = "What is a transformer?"
retrieved_docs = faiss_db.similarity_search(question, k=5)
context = "".join(doc.page_content + "\n" for doc in retrieved_docs)
print(context)

In [ ]:
torch.cuda.empty_cache()
print(inference(question=question, context=context))

In [ ]:
print("For this answer I used the following documents:")
for doc in retrieved_docs:
    print(doc.metadata)